In [39]:
import pandas as pd
import os
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus

In [2]:
df_raw = pd.read_csv('../data/31304.csv', sep=';', thousands='.', decimal=',')

In [3]:
df_raw.head()

,Sexo,Edad,Provincias,Periodo,Total
0,Ambos sexos,Total,Total Nacional,1 de julio de 2022,47615034.0
1,Ambos sexos,Total,Total Nacional,1 de enero de 2022,47432893.0
2,Ambos sexos,Total,Total Nacional,1 de julio de 2021,47331302.0
3,Ambos sexos,Total,Total Nacional,1 de enero de 2021,47398695.0
4,Ambos sexos,Total,Total Nacional,1 de julio de 2020,47355685.0


In [4]:
print(df_raw.dtypes)

Sexo           object
Edad           object
Provincias     object
Periodo        object
Total         float64
dtype: object


In [5]:
df_raw['Sexo'].unique()

array(['Ambos sexos', 'Hombres', 'Mujeres'], dtype=object)

In [6]:
df_raw['Edad'].unique()

array(['Total', '0 años', '1 año', '2 años', '3 años', '4 años', '5 años',
       '6 años', '7 años', '8 años', '9 años', '10 años', '11 años',
       '12 años', '13 años', '14 años', '15 años', '16 años', '17 años',
       '18 años', '19 años', '20 años', '21 años', '22 años', '23 años',
       '24 años', '25 años', '26 años', '27 años', '28 años', '29 años',
       '30 años', '31 años', '32 años', '33 años', '34 años', '35 años',
       '36 años', '37 años', '38 años', '39 años', '40 años', '41 años',
       '42 años', '43 años', '44 años', '45 años', '46 años', '47 años',
       '48 años', '49 años', '50 años', '51 años', '52 años', '53 años',
       '54 años', '55 años', '56 años', '57 años', '58 años', '59 años',
       '60 años', '61 años', '62 años', '63 años', '64 años', '65 años',
       '66 años', '67 años', '68 años', '69 años', '70 años', '71 años',
       '72 años', '73 años', '74 años', '75 años', '76 años', '77 años',
       '78 años', '79 años', '80 años', '81 años', '8

In [7]:
df_raw['Provincias'].unique()

array(['Total Nacional', '02 Albacete', '03 Alicante/Alacant',
       '04 Almería', '01 Araba/Álava', '33 Asturias', '05 Ávila',
       '06 Badajoz', '07 Balears, Illes', '08 Barcelona', '48 Bizkaia',
       '09 Burgos', '10 Cáceres', '11 Cádiz', '39 Cantabria',
       '12 Castellón/Castelló', '13 Ciudad Real', '14 Córdoba',
       '15 Coruña, A', '16 Cuenca', '20 Gipuzkoa', '17 Girona',
       '18 Granada', '19 Guadalajara', '21 Huelva', '22 Huesca',
       '23 Jaén', '24 León', '25 Lleida', '27 Lugo', '28 Madrid',
       '29 Málaga', '30 Murcia', '31 Navarra', '32 Ourense',
       '34 Palencia', '35 Palmas, Las', '36 Pontevedra', '26 Rioja, La',
       '37 Salamanca', '38 Santa Cruz de Tenerife', '40 Segovia',
       '41 Sevilla', '42 Soria', '43 Tarragona', '44 Teruel', '45 Toledo',
       '46 Valencia/València', '47 Valladolid', '49 Zamora',
       '50 Zaragoza', '51 Ceuta', '52 Melilla'], dtype=object)

In [8]:
df_raw['Periodo'].unique()

array(['1 de julio de 2022', '1 de enero de 2022', '1 de julio de 2021',
       '1 de enero de 2021', '1 de julio de 2020', '1 de enero de 2020',
       '1 de julio de 2019', '1 de enero de 2019', '1 de julio de 2018',
       '1 de enero de 2018', '1 de julio de 2017', '1 de enero de 2017',
       '1 de julio de 2016', '1 de enero de 2016', '1 de julio de 2015',
       '1 de enero de 2015', '1 de julio de 2014', '1 de enero de 2014',
       '1 de julio de 2013', '1 de enero de 2013', '1 de julio de 2012',
       '1 de enero de 2012', '1 de julio de 2011', '1 de enero de 2011',
       '1 de julio de 2010', '1 de enero de 2010', '1 de julio de 2009',
       '1 de enero de 2009', '1 de julio de 2008', '1 de enero de 2008',
       '1 de julio de 2007', '1 de enero de 2007', '1 de julio de 2006',
       '1 de enero de 2006', '1 de julio de 2005', '1 de enero de 2005',
       '1 de julio de 2004', '1 de enero de 2004', '1 de julio de 2003',
       '1 de enero de 2003', '1 de julio de 2002', 

In [9]:
# Filtrar sexos individuales
df = df_raw[df_raw["Sexo"].isin(["Hombres", "Mujeres"])].copy()

In [10]:
# Eliminar filas con edades agregadas
df = df[~df["Edad"].isin(["Total", "85 y más años", "100 y más años"])]

In [11]:
# --- Transformar campos ---
# Sexo a una sola letra
df["sex"] = df["Sexo"].map({"Hombres": "M", "Mujeres": "F"})

# Edad a entero
df["age"] = df["Edad"].str.extract("(\d+)").astype(int)

In [12]:
# Extraer año
df["year"] = df["Periodo"].str.extract(r"(\d{4})").astype(int)

In [13]:
# Extraer código de provincia
df["code_ine_prov"] = df["Provincias"].str.extract(r"^(\d{2})")

In [14]:
# Asegurar que code_ine_prov es string de 2 dígitos
df["code_ine_prov"] = df["code_ine_prov"].astype(str).str.zfill(2)

In [41]:
# --- Cargar tabla de CCAA ---
df_codigos_ccaa = pd.read_csv("../data/codigos_ine_prov_ccaa/codigos_ccaa_sin_CeutaMelilla.csv", delimiter= ';', dtype={"cod_ccaa": str})
df_codigos_ccaa.rename(columns={"cod_ccaa": "code_ine_ccaa", "name": "name_ccaa"}, inplace=True)

In [42]:
df_codigos_ccaa.head()

,code_ine_ccaa,name_ccaa
0,01,Andalucía
1,02,Aragón
2,03,Asturias
3,04,Islas Baleares
4,05,Islas Canarias


In [43]:
# --- Cargar tabla de provincias ---
df_codigos_prov = pd.read_csv("../data/codigos_ine_prov_ccaa/codigos_prov_mapa.csv", delimiter= ';', dtype={"cod_prov": str})
df_codigos_prov.rename(columns={"cod_prov": "code_ine_prov", "name": "name_prov"}, inplace=True)

In [44]:
df_codigos_prov.head()

,code_ine_prov,name_prov
0,01,Álava
1,02,Albacete
2,03,Alicante
3,04,Almería
4,05,Ávila


In [45]:
df_prov_ccaa = pd.read_csv("../data/codigos_ine_prov_ccaa/mapeo_codigos.csv", delimiter= ';', dtype={"cod_prov": str, "cod_ccaa": str})
df_prov_ccaa.rename(columns={"cod_prov": "code_ine_prov", "cod_ccaa": "code_ine_ccaa"}, inplace=True)
df_prov_ccaa.head()

,code_ine_prov,code_ine_ccaa
0,33,03
1,39,06
2,15,12
3,27,12
4,32,12


In [46]:
# 3) Hacer un merge de df1 con df2_ren sobre 'cod_ine_prov'
df_temp = df_prov_ccaa.merge(df_codigos_prov, on='code_ine_prov', how='left')
# 4) A continuación, hacer el merge con df3_ren sobre 'cod_ine_ccaa'
df_codigos_final = df_temp.merge(df_codigos_ccaa, on='code_ine_ccaa', how='left')

In [47]:
df_codigos_final

,code_ine_prov,code_ine_ccaa,name_prov,name_ccaa
0,33,03,Asturias,Asturias
1,39,06,Cantabria,Cantabria
2,15,12,A Coruña,Galicia
3,27,12,Lugo,Galicia
4,32,12,Ourense,Galicia
5,36,12,Pontevedra,Galicia
6,22,02,Huesca,Aragón
7,44,02,Teruel,Aragón
8,50,02,Zaragoza,Aragón
9,31,15,Navarra,Navarra


In [48]:
# 1) Asegúrate de que todas las columnas sean string
#df_codigos_final = df_codigos_final.astype(str)

# 2) Guardar en CSV:
#df_codigos_final.to_csv('mapeo_cod_ine_prov_ccaa_names.csv',
#                sep=',',          # separador de campos
#                index=False,      # no incluir índice como columna extra
#                encoding='utf-8') # codificación UTF-8


In [49]:
# Renombrar población
df.rename(columns={"Total": "population"}, inplace=True)

In [50]:
df.sample()

,Sexo,Edad,Provincias,Periodo,population,sex,age,year,code_ine_prov
1630874,Mujeres,88 años,45 Toledo,1 de julio de 1997,NaN,F,88,1997,45


In [51]:
df.head()

,Sexo,Edad,Provincias,Periodo,population,sex,age,year,code_ine_prov
573248,Hombres,0 años,Total Nacional,1 de julio de 2022,170022.5,M,0,2022,nan
573249,Hombres,0 años,Total Nacional,1 de enero de 2022,174194.0,M,0,2022,nan
573250,Hombres,0 años,Total Nacional,1 de julio de 2021,170071.0,M,0,2021,nan
573251,Hombres,0 años,Total Nacional,1 de enero de 2021,174641.0,M,0,2021,nan
573252,Hombres,0 años,Total Nacional,1 de julio de 2020,180200.0,M,0,2020,nan


In [52]:
print(df.dtypes)

Sexo              object
Edad              object
Provincias        object
Periodo           object
population       float64
sex               object
age                int64
year               int64
code_ine_prov     object
dtype: object


In [53]:
# Aseguramos que ambos DataFrames tengan el mismo tipo en la columna de código de provincia:
df['code_ine_prov']   = df['code_ine_prov'].astype(str)
df_codigos_final['code_ine_prov'] = df_codigos_final['code_ine_prov'].astype(str)

In [54]:
# 2. Seleccionamos solo las columnas que necesitas de df_final (para no arrastrar nada extra):
cols_para_merge = ['code_ine_prov', 'code_ine_ccaa', 'name_prov', 'name_ccaa']
df_mergeable   = df_codigos_final[cols_para_merge]

In [55]:
# 3. Haz el merge “left” de df con df_mergeable, emparejando por el código de provincia:
df_completo = df.merge(
    df_mergeable,
    left_on='code_ine_prov',
    right_on='code_ine_prov',
    how='left'
)

In [56]:
df_completo.sample()

,Sexo,Edad,Provincias,Periodo,population,sex,age,year,code_ine_prov,code_ine_ccaa,name_prov,name_ccaa
163384,Hombres,29 años,32 Ourense,1 de julio de 2022,1395.969132,M,29,2022,32,12,Ourense,Galicia


In [57]:
# Seleccionar columnas finales
df_poblacion_estructurado_tab_31304 = df_completo[[
    "code_ine_ccaa", "name_ccaa",
    "code_ine_prov", "name_prov",
    "year", "age", "sex", "population"
]].copy()

In [58]:
# [fuente]_[tema]_[detalle]_[versión]
#df_poblacion_estructurado_tab_31304.to_csv('mongo_exportable/INE_31304_poblacion_raw_v2.csv',
#                sep=',',          # separador de campos
#                index=False,      # no incluir índice como columna extra
#                encoding='utf-8') # codificación UTF-8

In [59]:
print(df_poblacion_estructurado_tab_31304.dtypes) 

code_ine_ccaa     object
name_ccaa         object
code_ine_prov     object
name_prov         object
year               int64
age                int64
sex               object
population       float64
dtype: object


In [60]:
for columuna in df_poblacion_estructurado_tab_31304:
    print(columuna)
    print(df_poblacion_estructurado_tab_31304[columuna].unique())
    print('----')

code_ine_ccaa
[nan '08' '10' '01' '16' '03' '07' '11' '04' '09' '06' '12' '02' '13' '14'
 '15' '05' '17' '18' '19']
----
name_ccaa
[nan 'Castilla-La Mancha' 'Valencia' 'Andalucía' 'País Vasco' 'Asturias'
 'Castilla y León' 'Extremadura' 'Islas Baleares' 'Cataluña' 'Cantabria'
 'Galicia' 'Aragón' 'Madrid' 'Murcia' 'Navarra' 'Islas Canarias'
 'La Rioja']
----
code_ine_prov
['nan' '02' '03' '04' '01' '33' '05' '06' '07' '08' '48' '09' '10' '11'
 '39' '12' '13' '14' '15' '16' '20' '17' '18' '19' '21' '22' '23' '24'
 '25' '27' '28' '29' '30' '31' '32' '34' '35' '36' '26' '37' '38' '40'
 '41' '42' '43' '44' '45' '46' '47' '49' '50' '51' '52']
----
name_prov
[nan 'Albacete' 'Alicante' 'Almería' 'Álava' 'Asturias' 'Ávila' 'Badajoz'
 'Baleares' 'Barcelona' 'Bizkaia' 'Burgos' 'Cáceres' 'Cádiz' 'Cantabria'
 'Castellón' 'Ciudad Real' 'Córdoba' 'A Coruña' 'Cuenca' 'Gipuzkoa'
 'Girona' 'Granada' 'Guadalajara' 'Huelva' 'Huesca' 'Jaén' 'León' 'Lleida'
 'Lugo' 'Madrid' 'Málaga' 'Murcia' 'Navarra' 'Oure

In [40]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [ ]:
coll = db['INE_31304_POBLACION_RAW']
coll.drop()
coll.insert_many(df_poblacion_estructurado_tab_31304.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_poblacion_estructurado_tab_31304))